# Agent 的上下文记忆

In [1]:
import shutup
shutup.please()

from langchain.chat_models import ChatOpenAI
from dotenv import load_dotenv
from langchain.tools import Tool
from langchain.chains import LLMRequestsChain, LLMChain
from langchain.prompts import Prompt
from langchain.memory import ConversationBufferMemory
from langchain.agents import ZeroShotAgent, AgentExecutor  # 有记忆功能的agent

load_dotenv()
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=0)

def generic_func(query):
    prompt = Prompt.from_template('回答问题：{query}')
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    return llm_chain.run(query)

def search_func(query):
    prompt = Prompt.from_template('''
    请根据以下搜索结果，回答用户问题。
    搜索结果：
    {requests_result}
    问题：{query}
    ''')
    llm_chain = LLMChain(llm=llm, prompt=prompt)
    llm_request_chain = LLMRequestsChain(llm_chain=llm_chain)
    input = {
        'query': query,
        'url': 'https://www.google.com/search?q='+query.replace(' ', '+')
    }
    return llm_request_chain.run(input)

tools = [
    Tool(
        name='通用大模型',
        func=generic_func,
        description='利用大模型自身能力，回答问题。'
    ),
    Tool(
        name='搜索引擎',
        func=search_func,
        description='其他模型没有正确答案时，使用该工具。'
    ),
]

加入提示词模板，让llm用中文回复，并且可以使用tools，这个模板词槽（input, chat_history等）是固定的，prefix是可以修改的。

In [2]:
suffix = """Begin!

{chat_history}
Question: {input}
{agent_scratchpad}"""

In [3]:
agent_prompt = ZeroShotAgent.create_prompt(
    tools=tools,
    prefix='请用中文回答以下问题，可以使用以下工具：',
    suffix=suffix,
    input_variables=['chat_history', 'input', 'agent_scratchpad']
)

llm_chain = LLMChain(llm=llm, prompt=agent_prompt, verbose=True)

为Agent加入记忆模块，我们用一种最通用的Agent类型：ZeroShotAgent

In [5]:
memory = ConversationBufferMemory(memory_key='chat_history')  # 和提示词模板中的{chat_history}应该一致
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools)       # 传的是llm chain而不是原llm
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, memory=memory, verbose=True)

result = agent_chain.run("帮我搜索一下双城之战第二季什么时候开播？")
result = agent_chain.run("有哪些主角？")
result



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
请用中文回答以下问题，可以使用以下工具：

通用大模型: 利用大模型自身能力，回答问题。
搜索引擎: 其他模型没有正确答案时，使用该工具。

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [通用大模型, 搜索引擎]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!


Question: 帮我搜索一下双城之战第二季什么时候开播？


> Finished chain.
Thought: 可以使用搜索引擎来获取这个信息。
Action: 搜索引擎
Action Input: 双城之战第二季开播时间
Observation: 回答：《双城之战第二季》将于2024年11月9日在腾讯视频开播。
Thought:

> Entering new LLMChain chain...
Prompt after formatting:
请用中文回答以下问题，可以使用以下工具：

通用大模型: 利用大模型自身能力，回答问题。
搜索引擎: 其他模型没有正确答案时，使用该工具。

Use the following format:

Question: the input question you must answer
Thought: you should 

'主角是凯特琳和蔚。'

可见提示词里有：this Thought/Action/Action Input/Observation can repeat N times，即一个tool效果不好的时候，可以尝试多个其他tool